### Dataset Prep:

In [1]:
!pip install datasets transformers[sentencepiece] evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "sst2")


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [3]:

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
dataset = load_dataset("imdb", split="train[:1000]")  # Load IMDB dataset (subset for demo)
checkpoint="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
datacollator = DataCollatorWithPadding(tokenizer=tokenizer)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [4]:

def tokenize_function(example):
  tokenized_example=tokenizer(example['sentence'],truncation=True)
  return tokenized_example

tokenized_datasets=raw_datasets.map(tokenize_function )

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

### Training

In [8]:
import evaluate

def compute_metrics(eval_pred):
  metrics=evaluate.load("glue","sst2")
  logits, labels=eval_pred
  predictions=np.argmax(logits, axis=-1)
  return metrics.compute(predictions=predictions, references=labels)


In [9]:
# Step1: Define TrainingArguments class. it will contain all the hyperparameters that
# Trainer will use for training and evaluation
from transformers import TrainingArguments

training_args=TrainingArguments("/content/drive/MyDrive/huggingface/model_trained/sequence_classification/glue_sst2")

# Step2: Define the model

from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We have recived the above warning because BERT is not pretrained for sequence classification task (classifying pair of sentences).
Warning indicates that head of the pretrained model has been discarded and a new suitable head is add for sequence classification. Some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It forces us to train those weights/parameters for the task at hand.

In [11]:
# Step3: initialize the Trainer class, providing following arguments
# 1. model
# 2. training_args
# 3. training dataset and validation dataset
# 4. datacollator
# 5. tokenizer
from transformers import Trainer
trainer=Trainer(model,
                training_args,
                train_dataset=tokenized_datasets['train'],
                eval_dataset=tokenized_datasets['validation'],
                data_collator=datacollator,  # We can skip this line because when we pass the tokenizer as processing_class arguemnt, the default data_collator is DataCollatorForPadding
                processing_class=tokenizer,
                compute_metrics=compute_metrics

                )



In [12]:
# Step4: start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.418900
1000,0.369600
1500,0.364600
2000,0.342300
2500,0.340900
3000,0.323400
3500,0.310900
4000,0.315300
4500,0.309100
5000,0.306500


TrainOutput(global_step=25257, training_loss=0.2125808878032169, metrics={'train_runtime': 2981.8228, 'train_samples_per_second': 67.76, 'train_steps_per_second': 8.47, 'total_flos': 3082513027395900.0, 'train_loss': 0.2125808878032169, 'epoch': 3.0})